In [1]:
import os
import sys 
from flask import Flask
from flask import request
from flask_api import status
import psycopg2
import json
import copy
from IPython.core.display import display, HTML

display(HTML("<style>.container { width:100% !important; }</style>"))
datosBaseDeDatos = {"usuario":"postgres","password":"carlos98","Host":"127.0.0.1","puerto":"5432","DB":"chatblocks"}

In [2]:
def abrirJson(name,path=os.getcwd()+"/" ):
    with open(path+name,encoding='utf-8') as data_file:  
        diccionario = json.load(data_file)
    return diccionario

In [3]:
def conexionBaseDeDatos(datosConexion):
    try:
        connection = psycopg2.connect(user = datosConexion["usuario"],
                              password = datosConexion["password"],
                              host = datosConexion["Host"],
                              port = datosConexion["puerto"],
                              database = datosConexion["DB"])    
        return connection
    except(Exception, psycopg2.Error) as error:
        print("Error al conectar con base de datos", error)

In [4]:
def consultar(connection,datos=False,consulta=False,tabla=False):
    #print("datos",datos)
    consultaBots = list()
    column_names = []
    data_rows = []
    diccionarioDatos = dict()
    consultas = list()
    cursor = connection.cursor()  
    cosulta =""    
    if datos:
        print("intenatando consulta...")
        if tabla == 'robots':
            consulta = "select * from robots where id_face = '"+datos["id_face"]+"';"
        else:
            consulta = "select * from "+tabla+" where "        
            i=0
            for dato, valorDato in datos.items():
                if i==0:
                    consulta = consulta+dato+"= '"+str(valorDato)+"'"                
                else:             
                    consulta = consulta+" and "+dato+"= '"+str(valorDato)+"'" 
                i=i+1
            #print(dato,":",valorDato)
            consulta= consulta + " ;" 
            #print(consulta)        
            #consulta = "select * from robots where name_robot = '"+datosBot["name_robot"]+"' and id_face = '"+ datosBot["id_face"]+"' and  type_blocki = '"+datosBot["type_blocki"]+"' and access_token= '"+datosBot["access_token"]+"' and id_user = '"+datosBot["id_user"]+"';"
    print("consulta:",consulta)
    cursor.execute(consulta)    
    column_names = [desc[0] for desc in cursor.description]  
    rows = cursor.fetchall()
    for row in rows:
        diccionarioDatos = dict()
        for i in range(len(column_names)):
            diccionarioDatos.update({column_names[i]:row[i]})
        consultaBots.append(diccionarioDatos)
    cursor.close()
    connection.close()  
    
    if len(consultaBots)==1:
        return consultaBots[0]
    elif len(consultaBots)==0:
        print('La consulta no trajo nada')
        return {"error": "true"}
    else:
        return consultaBots

In [5]:
def insert(datos,tabla,connection):
    cursor = connection.cursor()
    values = ""
    datosInsert = ""
    i = 0;
    for dato, valorDato in datos.items():
        if i==0:
            values = values+dato 
            datosInsert = datosInsert+"'"+str(valorDato)+"' "                
        else:
            values = values+","+dato
            datosInsert =datosInsert+",'"+str(valorDato)+"' "
        i=i+1
    insersion  = "insert into "+tabla+" ( "+values+" ) values ("+datosInsert+" );" 
    cursor.execute(insersion)
    connection.commit()
    cursor.close()
    connection.close()
    return datos

In [6]:
def update(datosActualizar,datos,tabla,connection):
    actualizar =" "
    actualizar = "update "+tabla+" set"
    j = 0
    for dato,informacionDato in datosActualizar.items():
        print(dato)
        if j==0:
            actualizar=actualizar+" "+dato+"="+str(informacionDato)
            j=j+1
        else:
             actualizar=actualizar+", "+dato+"="+str(informacionDato)
    
    i=0
    actualizar=actualizar+" where "
    for dato, valorDato in datos.items():
        if i==0:
            actualizar = actualizar+dato+"= '"+str(valorDato)+"'"                
        else:             
            actualizar = actualizar+" and "+dato+"= '"+str(valorDato)+"'" 
        i=i+1
    actualizar= actualizar + " ;"  

In [7]:
app = Flask(__name__)
@app.route('/access_token', methods = ['POST'])
def postAccessToken():
    try:         
        connection = conexionBaseDeDatos(datosBaseDeDatos)        
        datosVariable = request.get_json()   
        tabla = "robots"
        con = consultaBots(connection,datos=datosVariable,tabla=tabla)        
        listaAcessToken=list()
        if type(con)==list:
            for i in range(len(con)):
                print(con[i])
                listaAcessToken.append(con[i]["access_token"])
        else:
            listaAcessToken.append(con["access_token"])         
        print(listaAcessToken)
        return {"access_token":listaAcessToken}
    except:
        return datosVariable,status.HTTP_404_NOT_FOUND


In [8]:
@app.route('/insert/<tabla>',methods=['POST'])
def funcionalidad(tabla):
    try:
        connection = conexionBaseDeDatos(datosBaseDeDatos)
        datos = request.get_json()
        print("Datos recibidos INSERT:")
        print(datos)
        insert(datos,str(tabla),connection)
        connection = conexionBaseDeDatos(datosBaseDeDatos)
        consulta = consultar(connection,datos=datos,tabla=str(tabla))
        print("Datos enviados INSERT:")
        print (consulta)
        return json.dumps(consulta)
    except (Exception, psycopg2.Error) as error :
        print("error",error)
        return tabla,status.HTTP_404_NOT_FOUND

In [9]:
@app.route('/select/<tabla>',methods=['GET'])
def select(tabla):
    try:
        connection = conexionBaseDeDatos(datosBaseDeDatos)
        datos = request.get_json()
        print("Datos recibidos SELECT:")
        print(datos)
        
        consulta = consultar(connection,datos=datos,tabla=str(tabla))
        print("Datos enviados SELECT:")
        print (consulta)
        return json.dumps(consulta)
    except (Exception, psycopg2.Error) as error :
        print("error",error)
        return tabla,status.HTTP_404_NOT_FOUND

In [ ]:
if __name__ == "__main__":    
    app.run(host='localhost',port='8181')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:8181/ (Press CTRL+C to quit)
127.0.0.1 - - [07/Jul/2021 14:53:13] "GET /select/robots HTTP/1.1" 200 -


Datos recibidos SELECT:
{'name_robot': 'Oswa1', 'id_face': '106842278334918', 'block_ini': 'Informativo | 2edb2c69-beb8-471f-9799-940599824c36 | oIgvWKILDrYMYDjlgAIk', 'type_blocki': 'informativo', 'access_token': 'EAAFMtaIPwugBAIRPT4mTm2a9nei1940NrJqMbhvvyPwfZAUZA6Id2YaXUeSu06TENy0QoDsprATI0J2ZCGHS2X6UQ4TG2AWkIbNQ3LLGdRMORJZBZBT9BhB58QusrYaXG90RiZAog3r9THJCXmX1m7vjArQ3KoX3PrEGYehS4j6TSblsfU73uf', 'id_user': 'charlygoldcruz@gmail.com', 'api_nlp': 'http://localhost:8282/'}
intenatando consulta...
consulta: select * from robots where id_face = '106842278334918';
La consulta no trajo nada
Datos enviados SELECT:
{'error': 'true'}
Datos recibidos INSERT:
{'name_robot': 'Oswa1', 'id_face': '106842278334918', 'block_ini': 'Informativo | 2edb2c69-beb8-471f-9799-940599824c36 | oIgvWKILDrYMYDjlgAIk', 'type_blocki': 'informativo', 'access_token': 'EAAFMtaIPwugBAIRPT4mTm2a9nei1940NrJqMbhvvyPwfZAUZA6Id2YaXUeSu06TENy0QoDsprATI0J2ZCGHS2X6UQ4TG2AWkIbNQ3LLGdRMORJZBZBT9BhB58QusrYaXG90RiZAog3r9THJCXmX1m7

127.0.0.1 - - [07/Jul/2021 14:53:13] "POST /insert/robots HTTP/1.1" 200 -
127.0.0.1 - - [07/Jul/2021 14:53:13] "POST /insert/bloqueinformativo HTTP/1.1" 200 -


intenatando consulta...
consulta: select * from robots where id_face = '106842278334918';
Datos enviados INSERT:
{'name_robot': 'Oswa1', 'id_face': '106842278334918', 'block_ini': 'Informativo | 2edb2c69-beb8-471f-9799-940599824c36 | oIgvWKILDrYMYDjlgAIk', 'type_blocki': 'informativo', 'access_token': 'EAAFMtaIPwugBAIRPT4mTm2a9nei1940NrJqMbhvvyPwfZAUZA6Id2YaXUeSu06TENy0QoDsprATI0J2ZCGHS2X6UQ4TG2AWkIbNQ3LLGdRMORJZBZBT9BhB58QusrYaXG90RiZAog3r9THJCXmX1m7vjArQ3KoX3PrEGYehS4j6TSblsfU73uf', 'api_nlp': 'http://localhost:8282/', 'id_robot': 583, 'id_user': 'charlygoldcruz@gmail.com'}
Datos recibidos INSERT:
{'next_id': 'Informativo | c65fe24d-e9c2-4b86-b07f-9f48db28e95a | oIgvWKILDrYMYDjlgAIk', 'namestate': 'Informativo | 2edb2c69-beb8-471f-9799-940599824c36 | oIgvWKILDrYMYDjlgAIk', 'contenido': 'Hola oswaldo', 'contenttype': 'text', 'typingtime': '1', 'id_robot': 583, 'blocktype': 'informativo'}
intenatando consulta...
consulta: select * from bloqueinformativo where next_id= 'Informativo | c6

...

Datos enviados INSERT:
{'namestate': 'Informativo | e61376a2-83b0-4ccd-90b2-0d3b8d9fc04b | 46973551-1467-4450-7433-103925931451 | XTHyJxz6aPUIFVONQdUE', 'contenido': 'Editando Bloque ', 'next_id': 'Input | 36f8147d-fcbb-41b0-8017-c8a481cbf661 | 46973551-1467-4450-7433-103925931451 | XTHyJxz6aPUIFVONQdUE', 'blocktype': 'informativo', 'contenttype': 'text', 'typingtime': '1', 'id_block': 4178, 'id_robot': 584, 'opc_nextid': None, 'pos_y': None, 'pos_x': None, 'tag_active': None}
Datos recibidos INSERT:
{'name_var': 'variableInputInput_|_36f8147d-fcbb-41b0-8017-c8a481cbf6619310', 'opc_type': 'Variable', 'var': 0, 'opc_data': 'String', 'id_robot': 584}
intenatando consulta...
consulta: select * from variables where name_var= 'variableInputInput_|_36f8147d-fcbb-41b0-8017-c8a481cbf6619310' and opc_type= 'Variable' and var= '0' and opc_data= 'String' and id_robot= '584' ;
Datos enviados INSERT:
{'id_var': 2501, 'opc_type': 'Variable', 'opc_data': 'String', 'var': '0', 'id_robot': 584, 'name_v

127.0.0.1 - - [07/Jul/2021 15:23:42] "POST /insert/bloqueinformativo HTTP/1.1" 200 -
127.0.0.1 - - [07/Jul/2021 15:23:42] "POST /insert/bloqueinformativo HTTP/1.1" 200 -
127.0.0.1 - - [07/Jul/2021 15:23:42] "POST /insert/bloqueinformativo HTTP/1.1" 200 -


intenatando consulta...
consulta: select * from bloqueinformativo where next_id= 'Informativo | d2b4f80b-3a35-4f2a-a7a5-dfdbdcf7f918 | 68473870-5030-7316-6351-136576696036 | XTHyJxz6aPUIFVONQdUE' and namestate= 'Informativo | 98d6627d-10d5-4f8b-ab2e-c764a00bfb69 | 68473870-5030-7316-6351-136576696036 | XTHyJxz6aPUIFVONQdUE' and contenido= 'Empieza el segundo bloque ' and contenttype= 'text' and typingtime= '1' and id_robot= '584' and blocktype= 'informativo' ;
Datos enviados INSERT:
{'namestate': 'Informativo | 98d6627d-10d5-4f8b-ab2e-c764a00bfb69 | 68473870-5030-7316-6351-136576696036 | XTHyJxz6aPUIFVONQdUE', 'contenido': 'Empieza el segundo bloque ', 'next_id': 'Informativo | d2b4f80b-3a35-4f2a-a7a5-dfdbdcf7f918 | 68473870-5030-7316-6351-136576696036 | XTHyJxz6aPUIFVONQdUE', 'blocktype': 'informativo', 'contenttype': 'text', 'typingtime': '1', 'id_block': 4179, 'id_robot': 584, 'opc_nextid': None, 'pos_y': None, 'pos_x': None, 'tag_active': None}
Datos recibidos INSERT:
{'next_id': '

127.0.0.1 - - [07/Jul/2021 15:23:42] "POST /insert/bloqueinformativo HTTP/1.1" 200 -
127.0.0.1 - - [07/Jul/2021 15:23:42] "POST /insert/bloqueinformativo HTTP/1.1" 200 -


Datos enviados INSERT:
{'namestate': 'Informativo | 38f82c6d-a32e-4687-ae19-71aebb228e68 | XTHyJxz6aPUIFVONQdUE', 'contenido': 'Adios oswa', 'next_id': 'Salida', 'blocktype': 'informativo', 'contenttype': 'text', 'typingtime': '1', 'id_block': 4182, 'id_robot': 584, 'opc_nextid': None, 'pos_y': None, 'pos_x': None, 'tag_active': None}
Datos recibidos INSERT:
{'next_id': '', 'namestate': 'Salida', 'contenido': '', 'contenttype': 'text', 'typingtime': 3, 'id_robot': 584, 'blocktype': 'informativo'}
intenatando consulta...
consulta: select * from bloqueinformativo where next_id= '' and namestate= 'Salida' and contenido= '' and contenttype= 'text' and typingtime= '3' and id_robot= '584' and blocktype= 'informativo' ;
Datos enviados INSERT:
{'namestate': 'Salida', 'contenido': '', 'next_id': '', 'blocktype': 'informativo', 'contenttype': 'text', 'typingtime': '3', 'id_block': 4183, 'id_robot': 584, 'opc_nextid': None, 'pos_y': None, 'pos_x': None, 'tag_active': None}
